In [1]:
from __future__ import annotations

import math
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd


In [2]:
# -------------------------
# Logical type assignment
# -------------------------

def infer_logical_type(row: pd.Series) -> str:
    """
    Map raw dtype + parse_success from profiling into a logical type:
    'datetime', 'numeric', or 'categorical'.
    """
    # If it parses well as datetime, treat as datetime
    if row.get("parse_success", 0.0) >= 0.5:
        return "datetime"

    dtype_str = str(row.get("dtype", "")).lower()

    # Rough numeric check
    if any(x in dtype_str for x in ["int", "float", "double", "decimal"]):
        return "numeric"

    # Fallback
    return "categorical"


# -------------------------
# Type compatibility T
# -------------------------

TYPE_COMPAT_MATRIX: Dict[Tuple[str, str], float] = {}

def _init_type_compat_matrix():
    # Base matrix (you can tweak these weights if you like)
    base = {
        ("numeric", "numeric"): 1.0,
        ("datetime", "datetime"): 1.0,
        ("categorical", "categorical"): 1.0,
        ("numeric", "categorical"): 0.4,
        ("categorical", "numeric"): 0.4,
        ("numeric", "datetime"): 0.2,
        ("datetime", "numeric"): 0.2,
        ("categorical", "datetime"): 0.3,
        ("datetime", "categorical"): 0.3,
    }
    TYPE_COMPAT_MATRIX.update(base)

_init_type_compat_matrix()


def type_compat(t1: str, t2: str) -> float:
    """Soft type-compatibility T in [0,1]."""
    return TYPE_COMPAT_MATRIX.get((t1, t2), 0.0)


# -------------------------
# Coverage factor U
# -------------------------

def coverage_factor(dist_i: int, dist_k: int, overlap: int) -> float:
    """
    Coverage factor U based on distinct-value overlap.

    U = min( overlap/dist_i, overlap/dist_k ), with safe guards.
    """
    if overlap <= 0 or dist_i <= 0 or dist_k <= 0:
        return 0.0
    return float(min(overlap / dist_i, overlap / dist_k))


# -------------------------
# Value similarity R
# -------------------------

def jaccard_similarity(set_a, set_b) -> float:
    """Jaccard similarity for symbolic sets."""
    if not set_a and not set_b:
        return 1.0
    if not set_a or not set_b:
        return 0.0
    inter = len(set_a & set_b)
    union = len(set_a | set_b)
    if union == 0:
        return 0.0
    return inter / union


def js_divergence(p: np.ndarray, q: np.ndarray) -> float:
    """
    Jensen-Shannon divergence between two distributions p and q.
    p and q are non-negative and sum to 1.
    """
    p = np.asarray(p, dtype=float)
    q = np.asarray(q, dtype=float)
    p /= p.sum() if p.sum() > 0 else 1.0
    q /= q.sum() if q.sum() > 0 else 1.0

    m = 0.5 * (p + q)

    def kl_div(a, b):
        mask = (a > 0) & (b > 0)
        a = a[mask]
        b = b[mask]
        return np.sum(a * np.log2(a / b))

    return 0.5 * kl_div(p, m) + 0.5 * kl_div(q, m)


def ks_statistic(a: np.ndarray, b: np.ndarray) -> float:
    """
    Simple two-sample Kolmogorov-Smirnov statistic D in [0,1].
    """
    a = np.sort(a)
    b = np.sort(b)
    n, m = len(a), len(b)
    if n == 0 or m == 0:
        return 0.0

    i = j = 0
    d = 0.0
    while i < n and j < m:
        if a[i] <= b[j]:
            i += 1
        else:
            j += 1
        cdf_a = i / n
        cdf_b = j / m
        d = max(d, abs(cdf_a - cdf_b))
    return d


def numeric_similarity(arr_i: np.ndarray, arr_k: np.ndarray, n_bins: int = 20) -> float:
    """
    JS/KS blend for numeric-numeric columns.

    Returns R in [0,1].
    """
    arr_i = np.asarray(arr_i, dtype=float)
    arr_k = np.asarray(arr_k, dtype=float)

    if len(arr_i) == 0 or len(arr_k) == 0:
        return 0.0

    # Optional down-sampling for speed on huge arrays
    max_sample = 10000
    if len(arr_i) > max_sample:
        arr_i = np.random.choice(arr_i, size=max_sample, replace=False)
    if len(arr_k) > max_sample:
        arr_k = np.random.choice(arr_k, size=max_sample, replace=False)

    # Common bin edges
    lo = min(arr_i.min(), arr_k.min())
    hi = max(arr_i.max(), arr_k.max())
    if lo == hi:
        return 1.0  # identical constant columns

    hist_i, bins = np.histogram(arr_i, bins=n_bins, range=(lo, hi), density=False)
    hist_k, _ = np.histogram(arr_k, bins=bins, density=False)

    # JS divergence -> similarity
    js = js_divergence(hist_i, hist_k)
    js = min(1.0, max(0.0, js))
    js_sim = 1.0 - js

    # KS statistic -> similarity
    ks = ks_statistic(arr_i, arr_k)
    ks = min(1.0, max(0.0, ks))
    ks_sim = 1.0 - ks

    return 0.5 * (js_sim + ks_sim)


In [3]:
@dataclass
class ColumnMeta:
    table: str
    column: str
    logical_type: str
    is_pk_candidate: bool
    is_dt_candidate: bool
    completeness: float
    distinctness: float
    distinct_count: int


def build_column_meta(df_profiles: pd.DataFrame) -> List[ColumnMeta]:
    meta_list: List[ColumnMeta] = []
    for _, row in df_profiles.iterrows():
        logical = infer_logical_type(row)
        meta_list.append(
            ColumnMeta(
                table=row["table"],
                column=row["column"],
                logical_type=logical,
                is_pk_candidate=bool(row.get("is_pk_candidate", False)),
                is_dt_candidate=bool(row.get("is_dt_candidate", False)),
                completeness=float(row.get("completeness", 0.0)),
                distinctness=float(row.get("distinctness", 0.0)),
                distinct_count=int(row.get("distinct_count", 0)),
            )
        )
    return meta_list


def load_tables(data_folder: str, table_names: List[str]) -> Dict[str, pd.DataFrame]:
    """
    Load each table from <data_folder>/<table>.csv
    (assuming file stem matches the 'table' column from profiling).
    """
    dfs: Dict[str, pd.DataFrame] = {}
    base = Path(data_folder)

    for t in sorted(set(table_names)):
        path = base / f"{t}.csv"
        if not path.exists():
            print(f"⚠️  Table file not found: {path}")
            continue
        try:
            df = pd.read_csv(path)
            dfs[t] = df
        except Exception as e:
            print(f"⚠️  Failed to read {path}: {e}")
    return dfs


def compute_relationship_scores(
    df_profiles: pd.DataFrame,
    data_folder: str,
    min_score: float = 0.0,
    require_pk_in_pair: bool = True,
) -> pd.DataFrame:
    """
    Main function to compute s(c_i, c_k) = T * Q * U * R for column pairs.

    Parameters
    ----------
    df_profiles : DataFrame
        Output from Section 3.1 (profiling).
    data_folder : str
        Folder containing the raw table CSVs.
    min_score : float
        Only keep pairs with s >= min_score.
    require_pk_in_pair : bool
        If True, only consider pairs where at least one column is a PK candidate.

    Returns
    -------
    DataFrame with one row per column pair and columns:
        - table_i, column_i, role_i, logical_type_i, ...
        - table_k, column_k, role_k, logical_type_k, ...
        - T, Q, U, R, overlap_count, s
    """
    # Build column metadata (primary key & datetime flags)
    meta_list = build_column_meta(df_profiles)

    # Load only tables that actually appear in profiling
    table_names = [m.table for m in meta_list]
    table_to_df = load_tables(data_folder, table_names)

    records = []

    # Pre-lookup: group metadata by table for speed
    by_table: Dict[str, List[ColumnMeta]] = {}
    for m in meta_list:
        by_table.setdefault(m.table, []).append(m)

    # Pre-calc mapping for (table, column) -> profile row (for completeness/distinctness)
    prof_index = df_profiles.set_index(["table", "column"])

    # Iterate over cross-table pairs
    tables = sorted(by_table.keys())
    for i_idx, t_i in enumerate(tables):
        for t_k in tables[i_idx + 1 :]:
            df_i = table_to_df.get(t_i)
            df_k = table_to_df.get(t_k)
            if df_i is None or df_k is None:
                continue

            cols_i = by_table[t_i]
            cols_k = by_table[t_k]

            for meta_i in cols_i:
                for meta_k in cols_k:
                    # Optional gate: at least one column must be a PK candidate
                    if require_pk_in_pair and not (meta_i.is_pk_candidate or meta_k.is_pk_candidate):
                        continue

                    # ---------------------------
                    # T: type compatibility
                    # ---------------------------
                    T_val = type_compat(meta_i.logical_type, meta_k.logical_type)
                    if T_val <= 0.0:
                        continue  # incompatible types, skip

                    # Q term: conservative column quality
                    qi = meta_i.completeness * meta_i.distinctness
                    qk = meta_k.completeness * meta_k.distinctness
                    Q_val = min(qi, qk)

                    # Get the raw Series for values
                    if meta_i.column not in df_i.columns or meta_k.column not in df_k.columns:
                        continue

                    s_i = df_i[meta_i.column].dropna()
                    s_k = df_k[meta_k.column].dropna()

                    # Evaluate distinct-value overlap sets
                    # Use strings for generality
                    set_i = set(map(str, s_i.unique()))
                    set_k = set(map(str, s_k.unique()))
                    overlap_count = len(set_i & set_k)

                    U_val = coverage_factor(meta_i.distinct_count, meta_k.distinct_count, overlap_count)
                    if overlap_count == 0 or U_val == 0.0:
                        continue  # no overlap, no relationship

                    # ---------------------------
                    # R: value similarity
                    # ---------------------------
                    if meta_i.logical_type == "numeric" and meta_k.logical_type == "numeric":
                        arr_i = pd.to_numeric(s_i, errors="coerce").dropna().to_numpy()
                        arr_k = pd.to_numeric(s_k, errors="coerce").dropna().to_numpy()
                        R_val = numeric_similarity(arr_i, arr_k)
                    else:
                        # Symbolic / mixed types -> Jaccard on strings
                        R_val = jaccard_similarity(set_i, set_k)

                    # Final score
                    s_val = T_val * Q_val * U_val * R_val

                    if s_val < min_score:
                        continue

                    role_i = ";".join(
                        r for r, flag in [
                            ("pk", meta_i.is_pk_candidate),
                            ("dt", meta_i.is_dt_candidate),
                        ]
                        if flag
                    ) or "other"

                    role_k = ";".join(
                        r for r, flag in [
                            ("pk", meta_k.is_pk_candidate),
                            ("dt", meta_k.is_dt_candidate),
                        ]
                        if flag
                    ) or "other"

                    records.append(
                        dict(
                            table_i=meta_i.table,
                            column_i=meta_i.column,
                            logical_type_i=meta_i.logical_type,
                            role_i=role_i,
                            completeness_i=meta_i.completeness,
                            distinctness_i=meta_i.distinctness,
                            distinct_count_i=meta_i.distinct_count,
                            table_k=meta_k.table,
                            column_k=meta_k.column,
                            logical_type_k=meta_k.logical_type,
                            role_k=role_k,
                            completeness_k=meta_k.completeness,
                            distinctness_k=meta_k.distinctness,
                            distinct_count_k=meta_k.distinct_count,
                            T=T_val,
                            Q=Q_val,
                            U=U_val,
                            R=R_val,
                            overlap_count=overlap_count,
                            s=s_val,
                        )
                    )

    rel_df = pd.DataFrame.from_records(records)
    return rel_df


In [4]:
# === USER INPUTS ===
# Path to profiling output from Section 3.1 (s_pk, s_dt, etc.)
profiling_csv_path = "C:\\Users\\joel\\Desktop\\CAiSE_2\\DGP_1\\data\\profile_primitives.csv"   # 👈 change this

# Folder containing the raw table CSVs
data_folder = "C:\\Users\\joel\\Desktop\\CAiSE_2\\DGP_1\\data\\food_deliver_clean"                                  # 👈 change this

# Where to write the relationship scores CSV
relationship_output_csv = "C:\\Users\\joel\\Desktop\\CAiSE_2\\DGP_1\\data\\relationship_scores.csv"   # 👈 change this



# # Path to profiling output from Section 3.1 (s_pk, s_dt, etc.)
# profiling_csv_path = "C:\\Users\\joel\\Desktop\\CAiSE_2\\TPC_H\\ProfilePrimitives_TPCH.csv"   # 👈 change this

# # Folder containing the raw table CSVs
# data_folder = "C:\\Users\\joel\\Desktop\\CAiSE_2\\TPC_H\\tpch_csv"                                  # 👈 change this

# # Where to write the relationship scores CSV
# relationship_output_csv = "C:\\Users\\joel\\Desktop\\CAiSE_2\\TPC_H\\relationship_scores.csv"   # 👈 change this



# Optional gates
MIN_SCORE = 0.0        # keep everything with s >= MIN_SCORE
REQUIRE_PK_IN_PAIR = True  # set False if you want all candidate pairs

# === RUN ===
df_profiles = pd.read_csv(profiling_csv_path)

relationship_df = compute_relationship_scores(
    df_profiles=df_profiles,
    data_folder=data_folder,
    min_score=MIN_SCORE,
    require_pk_in_pair=REQUIRE_PK_IN_PAIR,
)

print(f"Computed {len(relationship_df)} candidate relationships.")
relationship_df.to_csv(relationship_output_csv, index=False)
print(f"Relationship scores written to: {relationship_output_csv}")

relationship_df.head()


Computed 16 candidate relationships.
Relationship scores written to: C:\Users\joel\Desktop\CAiSE_2\DGP_1\data\relationship_scores.csv


,table_i,column_i,logical_type_i,role_i,completeness_i,distinctness_i,distinct_count_i,table_k,column_k,logical_type_k,role_k,completeness_k,distinctness_k,distinct_count_k,T,Q,U,R,overlap_count,s
0,Cancellations,OrigOrderKey,categorical,pk,1.0,1.0,3661,Orders,ORD_ID,categorical,pk,1.0,1.000000,9155,1.0,1.000000,0.399891,0.399891,3661,0.159913
1,CardUpdates,CustomerAlias,categorical,pk,1.0,1.0,3000,CardVerifications,CustJoinKey,categorical,pk,1.0,1.000000,3000,1.0,1.000000,1.000000,1.000000,3000,1.000000
2,CardUpdates,CustomerAlias,categorical,pk,1.0,1.0,3000,CustomerValidation,CustKey,categorical,pk,1.0,1.000000,3000,1.0,1.000000,1.000000,1.000000,3000,1.000000
3,CardUpdates,CustomerAlias,categorical,pk,1.0,1.0,3000,Customers,CustomerID,categorical,pk,1.0,1.000000,3000,1.0,1.000000,1.000000,1.000000,3000,1.000000
4,CardUpdates,CustomerAlias,categorical,pk,1.0,1.0,3000,Orders,CustRef,categorical,other,1.0,0.308247,2822,1.0,0.308247,0.940667,0.940667,2822,0.272753
